In [252]:
from selenium import webdriver
from selenium.webdriver.common.by import By
import time
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
import pandas as pd

In [216]:
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/87.0.4280.88 Safari/537.36'}

options=webdriver.EdgeOptions()
# Add the arguments for Edge
options.add_argument("-inprivate")  # For private browsing in Edge
options.add_argument("--disable-blink-features=AutomationControlled")
options.add_argument("--disable-cache")
options.add_argument("--disable-cookies")

In [217]:
url="https://super.walmart.com.mx/"
driver=webdriver.Edge(options=options)
driver.get(url)
time.sleep(5)

In [218]:
try:
    elemento = driver.find_element(By.NAME, 'select-store-button')
    
except NoSuchElementException:
    elemento = driver.find_element(By.CSS_SELECTOR, 'img[src="https://i5.walmartimages.com/dfw/4ff9c6c9-14f0/k2-_8ce34043-6050-4f39-afb2-3e76a554f8a3.v1.png"]')

elemento.click()

In [219]:
#Código postal
time.sleep(4)
cp=driver.find_element(By.CSS_SELECTOR,'input[data-automation-id="store-zip-code"]')
cp.clear()
cp.send_keys('06720')
time.sleep(4)
radio_buttons = driver.find_element(By.NAME,'pickup-store')
radio_buttons.click()
#Elegir
elegir=driver.find_element(By.CSS_SELECTOR,'button[data-automation-id="save-label"]')
elegir.click()

In [220]:
#Ingresar búsqueda
busca=driver.find_element(By.CSS_SELECTOR,'input[placeholder="Buscar en Walmart"]')
time.sleep(5)
busca.send_keys('limón')
busca.send_keys(Keys.ENTER)

In [221]:
#Obtener sopa
time.sleep(10)
html=driver.page_source


<html lang="es-MX" style=""><head><link href="//www.walmart.com.mx" rel="dns-prefetch"/><meta charset="utf-8"/><meta content="105223049547814" property="fb:app_id"/><meta content="width=device-width, initial-scale=1.0, minimum-scale=1, interactive-widget=resizes-content" name="viewport"/><link href="https://tap.walmart.com " rel="dns-prefetch"/><link as="font" crossorigin="anonymous" fetchpriority="high" href="https://i5.walmartimages.com/dfw/63fd9f59-a78c/fcfae9b6-2f69-4f89-beed-f0eeb4237946/v1/BogleWeb_subset-Bold.woff2" rel="preload" type="font/woff2"/><link as="font" crossorigin="anonymous" fetchpriority="high" href="https://i5.walmartimages.com/dfw/63fd9f59-a78c/fcfae9b6-2f69-4f89-beed-f0eeb4237946/v1/BogleWeb_subset-Regular.woff2" rel="preload" type="font/woff2"/><title>Walmart.com</title><meta content="Walmart.com" property="og:title"/><meta content="Walmart.com" name="description" property="og:description"/><meta content="https://i5.walmartimages.com.mx/gr/images/product-images

In [231]:
soup = BeautifulSoup(html, 'xml')

In [233]:
pretty_soup=soup.prettify()
pretty_soup 

'<?xml version="1.0" encoding="utf-8"?>\n<html lang="es-MX" style="">\n <head>\n  <link href="//www.walmart.com.mx" rel="dns-prefetch">\n   <meta charset="utf-8">\n    <meta content="105223049547814" property="fb:app_id">\n     <meta content="width=device-width, initial-scale=1.0, minimum-scale=1, interactive-widget=resizes-content" name="viewport">\n      <link href="https://tap.walmart.com " rel="dns-prefetch">\n       <link as="font" crossorigin="anonymous" fetchpriority="high" href="https://i5.walmartimages.com/dfw/63fd9f59-a78c/fcfae9b6-2f69-4f89-beed-f0eeb4237946/v1/BogleWeb_subset-Bold.woff2" rel="preload" type="font/woff2">\n        <link as="font" crossorigin="anonymous" fetchpriority="high" href="https://i5.walmartimages.com/dfw/63fd9f59-a78c/fcfae9b6-2f69-4f89-beed-f0eeb4237946/v1/BogleWeb_subset-Regular.woff2" rel="preload" type="font/woff2">\n         <title>\n          Walmart.com\n         </title>\n         <meta content="Walmart.com" property="og:title">\n          <me

In [250]:
span_tags = soup.select('div[data-automation-id="product-price"] span')
product_titles = [title.text for title in soup.select('span[data-automation-id="product-title"]')]

precios = []

for span in span_tags:
    text = span.text
    if 'precio actual' in text:
        precios.append(text.split("precio actual")[1].strip())


        

['Limón sin semilla por kilo',
 'Limón agrio por kilo',
 'Limón eureka por kilo',
 'Limón agrio extra en malla 1.3 kg',
 'Naranja importada por kilo',
 'Limón agrio en malla 1 kilo',
 'Limón orgánico Marketside 1.3 kg',
 'Té McCormick limón 27 sobres',
 'Sal con chile y limón McCormick 104 g',
 'Bebida hidratante Flashlyte coco - limón con electrolitos 625 ml',
 'Té de limón por pieza',
 'Concentrado de limón Mega Limón 1 l',
 'Mayonesa McCormick con limón 390 g',
 'Té  de limón Great Value 18 sobres 1.5 g c /u',
 'Té de limón La Pastora 20 sobres de 1.5 g c/u',
 'Mayonesa McCormick con limón 870 g',
 'Lavatrastes líquido Salvo Limón 750 ml',
 'Mandarina Marketside en malla 907 g',
 'Mayonesa McCormick con limón 228 g',
 'Mayonesa Heinz limón 390 g',
 'Gajos de Toronja Del Monte Fruit Naturals fruit naturals 184 g',
 'Té de Jengibre y Limón Great Value 18 sobres 1.5 g c /u',
 'Bolsa para Regalo Grande Limones Rosa',
 'Mayonesa McCormick con limón 320 g',
 'Mayonesa McCormick con limón 

In [253]:
#Llevar product_title y precios a un dataframe
df=pd.DataFrame({'proudcto':product_titles,'precio':precios})
df

,proudcto,precio
0,Limón sin semilla por kilo,$39.90/kg
1,Limón agrio por kilo,$39.90/kg
2,Limón eureka por kilo,$52.00/kg
3,Limón agrio extra en malla 1.3 kg,$52.00
4,Naranja importada por kilo,$32.90/kg
5,Limón agrio en malla 1 kilo,$37.90
6,Limón orgánico Marketside 1.3 kg,$48.90
7,Té McCormick limón 27 sobres,$25.00
8,Sal con chile y limón McCormick 104 g,$29.50
9,Bebida hidratante Flashlyte coco - limón con e...,$20.00
